In [15]:
import cv2, numpy as np, os

In [16]:
img_size = 60 # has to be the same as for training
#chromakey values for the color green
h,s,v,h1,s1,v1 = 48,92,0,64,255,255
#parameter for segmenting image
pad = 60

In [21]:
#finds the largest contour in a list of contours
#returns a single contour
def largest_contour(contours): return max(contours, key=cv2.contourArea)[1]

#finds the center of a contour
#takes a single contour
#returns (x,y) position of the contour
def contour_center(c):
    M = cv2.moments(c)
    try: center = int(M['m10']/M['m00']), int(M['m01']/M['m00'])
    except: center = 0,0
    return center

#takes image and range
#returns parts of image in range
def only_color(frame,(h,s,v,h1,s1,v1)):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
     
    lower1 = np.array([0,120,70], np.uint8)
    upper1 = np.array([10, 255, 255], np.uint8)
    mask1 = cv2.inRange(hsv, lower1, upper1)

    lower2 = np.array([170,120,70], np.uint8)
    upper2 = np.array([180, 255, 255], np.uint8)
    mask2 = cv2.inRange(hsv, lower2, upper2)

    mask = mask1+mask2
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    res = cv2.bitwise_and(frame, frame, mask = mask)
    
    return res, mask
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     lower, upper = np.array([h,s,v]), np.array([h1,s1,v1])
#     mask = cv2.inRange(hsv, lower, upper)
#     res = cv2.bitwise_and(img, img, mask=mask)
#     kernel = np.ones((3,3), np.uint)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
#     return res, mask

#returns the region of interest around the largest countour
#the accounts for objects not being centred in the frame
def bbox(img, c):
    x,y,w,h = cv2.boundingRect(c)
    return img[y-pad:y+h+pad, x-pad:w+x+pad], (x,y)


SyntaxError: invalid syntax (<ipython-input-21-29b021d8aebb>, line 16)

In [18]:
classes = { 1:'Circle',
            2:'Square',      
            3:'Triangle' }

In [19]:
from keras.models import load_model
__dirname = os.path.join(os.getcwd(),"../images/shapes/circle")
# print(__dirname)
iModel = load_model('../model/shapesmodel.h5')
dimData = np.prod([img_size, img_size])

In [20]:
from keras.preprocessing import image


def predict(filename):
    img  = cv2.imread(filename)
    print(img)
    imgc = img.copy()
    height, width, _ = img.shape
    
    print(img, imgc, img.shape)

    #mask of the green regions in the image    
    _, mask = only_color(img,(h,s,v,h1,s1,v1))

    #find the contours in the image
    _, contours, _  = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #iterate through the contours, "Keras, what shape is this contour?"
    for c in contours:
        #if the contour is too big or too small, it can be ignored
        area = cv2.contourArea(c)
        #print area
        if area> 3000 and area< 1180000:

            #crop out the green shape
            roi, coords = bbox(img, c)

            #filter out contours that are long and stringy
            if np.prod(roi.shape[:2])>10:

                #get the black and white image of the shape
                roi = cv2.resize(roi, (img_size, img_size))
                _, roi = only_color(roi, (h,s,v,h1,s1,v1))
                roi= 255-roi #Keras likes things black on white
                mask = cv2.resize(roi, (img_size, img_size))
                mask = mask.reshape(dimData)
                mask = mask.astype('float32')
                mask /=255
                #feed image into model
                prediction = iModel.predict(mask.reshape(1,dimData))[0].tolist()

                #create text --> go from categorical labels to the word for the shape.
                text = ''
                p_val, th = .25, .5
                if max(prediction)> p_val:
                    if prediction[0]>p_val and prediction[0]==max(prediction): text, th =  'triangle', prediction[0]
                    if prediction[1]>p_val and prediction[1]==max(prediction): text, th =  'star', prediction[1]
                    if prediction[2]>p_val and prediction[2]==max(prediction): text, th =  'square', prediction[2]
                    if prediction[3]>p_val and prediction[3]==max(prediction): text, th =  'circle', prediction[3]
                
                #draw the contour
                cv2.drawContours(imgc, c, -1, (0,0,255), 1)

                #draw the text
                org, font, color = (coords[0], coords[1]+int(area/400)), cv2.FONT_HERSHEY_SIMPLEX, (0,0,255)
                cv2.putText(imgc, text, org, font, int(2.2*area/15000), color, int(6*th), cv2.LINE_AA)

                #paste the black and white image onto the source image (picture in picture)
                if text!='': imgc[imgc.shape[0]-200:imgc.shape[0], img.shape[1]-200:img.shape[1]] = cv2.cvtColor(cv2.resize(roi, (200,200)), cv2.COLOR_GRAY2BGR)
                
    cv2.imshow('img', cv2.resize(imgc, (640, 480))) #expect 2 frames per second





# def predict(filename):
#     test_image = image.load_img(filename, target_size = (60,60))
#     test_image = image.img_to_array(test_image)
#     test_image = np.expand_dims(test_image, axis=0)
#     result = iModel.predict(test_image)[0]
#     test_image = np.squeeze(test_image)
#     plt.imshow(test_image/255.)
#     accuracy = np.nanmax(result)*100
#     print(classes[result.argmax()+1])
#     print("Accuracy: {:.2f} %".format(accuracy))
# #     print(result)
# #     print(result.argmax())
# #     print(np.nanmax(result))
# #     print(np.where(result == np.nanmax(result, axis = 1)))
# #     print(result.loc[result[0].argmax(), "SignName"])

In [9]:
import random 
filename = random.choice(os.listdir(__dirname))
# print(os.path.join(__dirname,filename))
predict(os.path.join(__dirname,filename))

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


TypeError: Expected Ptr<cv::UMat> for argument 'kernel'